# eQTL

This tutorial illustrates the use of limix to analyse expression datasets.
We consider gene expression levels from a yeast genetics
study with freely available data.
This data set span 109 individuals with 2,956 marker SNPs and expression
levels for 5,493 in glucose and ethanol growth media respectively.

## Importing limix

In [1]:
import limix

## Shorter output

In [2]:
import pandas as pd
pd.set_option("display.float_format", "{:8.5g}".format)

ValueError: Value must be a callable

## Downloading data

We are going to use a HDF5 file containg phenotype and genotyped data from
a remote repository.
Limix provides some handy utilities to perform common command line tasks,
like as downloading and extracting files.
However, feel free to use whatever method you prefer.

In [ ]:
url = "http://rest.s3for.me/limix/smith08.hdf5.bz2"
limix.sh.download(url, verbose=False, force=True)
print(limix.sh.filehash("smith08.hdf5.bz2"))

In [ ]:
limix.sh.extract("smith08.hdf5.bz2", verbose=False)
limix.io.hdf5.see("smith08.hdf5")

In [ ]:
data = limix.io.hdf5.read_limix("smith08.hdf5")
Y = data['phenotype']
G = data['genotype']
print(Y)

## Selecting gene YBR115C under the glucose condition

Query for a specific phenotype, select the phenotype itself, and plot it.
The glucose condition is given by the environment ``0``.

In [ ]:
y = Y[:, (Y.gene_ID == "YBR115C") & (Y.environment==0)]
y = y.stack(z=('sample', 'outcome')).reset_index('z')
y = y.rename(z="sample")
limix.plot.normal(y)
limix.plot.show()

## Genetic relatedness matrix

The genetic relatedness will be determined by the inner-product of SNP
readings between individuals, and the result will be visualised via heatmap.

In [ ]:
K = limix.stats.linear_kinship(G.values, verbose=False)
limix.plot.kinship(K)
limix.plot.show()

## Univariate association test with linear mixed model

You have the option to either pass a raw array of samples-by-candidates for
the association scan or pass a tabular structure naming those candidates.
We recommend the second option as it will help maintain the association between
the results and the corresponding candidates.

The naming of those candidates is defined here by concatenating the chromossome
name and base-pair position.
However, it is often the case that SNP IDs are provided along with the
data, which can naturally be used for naming those candidates.

In [ ]:
from pandas import DataFrame
import numpy as np

print(G)

As you can see, we now have a pandas data frame ``G`` that keeps the candidate
identifications together with the actual allele read.
This data frame can be readily used to perform association scan.

In [ ]:
print(y)

In [ ]:
print(G)

In [ ]:
qtl = limix.qtl.st_scan(G, y, 'normal', K, verbose=False)
print(qtl)

Inspecting the p-values and effect-sizes are now easier because candidate
names are kept together with their corresponding statistics.

In [ ]:
pv = qtl.variant_pvalues
pv = pv.sortby(pv).to_dataframe()
pv["-log10(pv)"] = -np.log10(pv["pv"])
print(pv.head())

In [ ]:
print(qtl.variant_effsizes.sel(candidate=pv.index).to_dataframe().head())

A Manhattan plot can help understand the result.

In [ ]:
limix.plot.manhattan(qtl.variant_pvalues)
limix.plot.show()

We then remove the temporary files.

In [ ]:
limix.sh.remove("smith08.hdf5.bz2")
limix.sh.remove("smith08.hdf5")